In [ ]:
#default_exp core

# Core

> Handle FHIR server communication and low-level resource functions.

In [ ]:
#export
import requests
from datetime import datetime, timezone
from uuid import uuid4

In [ ]:
import json

In [ ]:
#export
# TODO: fix this hack
request_headers = {}

In [ ]:
#export
def get_as_raw_json(api_base, resource_type, id_or_params=None):
    "GET FHIR resources of `resource_type` in JSON format"
    url = f'{api_base}/{resource_type}'
    params = dict(_format = 'json')
    if isinstance(id_or_params, dict): params = {**params, **id_or_params}
    elif isinstance(id_or_params, str):
        if id_or_params[0] in ['/','?']: raise Exception(f'invalid id_or_params {id_or_params}') # TODO: clean
        url += f'/{id_or_params}'
    response = requests.get(url, params, headers=request_headers)
    print('GET', response.url)
    return response.json()

- `api_base` a "real" API base like `http://hapi.fhir.org/baseR4` 
- `resource_type` the type of resource to get. e.g. `Medication`
- `url_suffix` the preferred way to add search criteria. e.g. `?subject=d28f9c95-8098-4794-b1d0-57e45faf2b39`

To search for some medications;
```
get_as_raw_json('https://server.fire.ly/r4', 'Medication')
```

To get a single medication;
```
get_as_raw_json('https://server.fire.ly/r4', Medication', '?_id=d28f9c95-8098-4794-b1d0-57e45faf2b39')
```

In [ ]:
#export
def get_next_as_raw_json(json_response):
    "GET the next set of results"
    for link in json_response['link']:
        if link['relation'] == 'next':
            url = link['url']
            print('GET',url)
            return requests.get(url, headers=request_headers).json()

In [ ]:
#export
def timestamp_now():
    return datetime.utcnow().strftime('%Y-%m-%dT%H:%M:%SZ')

In [ ]:
#export
def new_bundle(bundle_type='collection'):
    return dict(resourceType='Bundle', 
                id=str(uuid4()),
                type=bundle_type, 
                timestamp=timestamp_now(),
                entry=[])

In [ ]:
# this will fail if the bundle we create cannot be converted to JSON
json.dumps(new_bundle())

'{"resourceType": "Bundle", "id": "efd73f5b-1a76-44d1-a948-4a9add134921", "type": "collection", "timestamp": "2021-09-16T12:34:37Z", "entry": []}'

In [ ]:
#export
def new_list(title, status='current', mode='snapshot'):
    return dict(resourceType='List', 
                id=str(uuid4()),
                title=title,
                status=status, 
                mode=mode,
                date=timestamp_now(),
                entry=[])

In [ ]:
json.dumps(new_list(title='test list'))

'{"resourceType": "List", "id": "2b1c363a-47d6-40dc-99ff-7a6721a50a2a", "title": "test list", "status": "current", "mode": "snapshot", "date": "2021-09-16T12:34:37Z", "entry": []}'

In [ ]:
# Set the base URL of the FHIR server that we will use in tests
api_base, resource_type = 'http://hapi.fhir.org/baseR4', 'ResearchStudy'
# api_base, resource_type = 'https://r4.smarthealthit.org', 'MedicationRequest'

The following cell shows how `get_as_raw_json` and `get_next_as_raw_json` can be used to read paged search results.

In [ ]:
bundle = get_as_raw_json(api_base, resource_type)
print(json.dumps(bundle, indent=2)[:500], '...')

GET http://hapi.fhir.org/baseR4/ResearchStudy?_format=json
{
  "resourceType": "Bundle",
  "id": "8b604b8c-a8f4-4243-9c39-09175c6cb28d",
  "meta": {
    "lastUpdated": "2021-09-16T12:34:38.178+00:00"
  },
  "type": "searchset",
  "total": 57,
  "link": [
    {
      "relation": "self",
      "url": "http://hapi.fhir.org/baseR4/ResearchStudy?_format=json"
    },
    {
      "relation": "next",
      "url": "http://hapi.fhir.org/baseR4?_getpages=8b604b8c-a8f4-4243-9c39-09175c6cb28d&_getpagesoffset=20&_count=20&_format=json&_pretty=true&_bundletype=searchs ...


In [ ]:
json_response = get_as_raw_json(api_base, resource_type)
page_count = 1
while json_response is not None:    
    for i, entry in enumerate(json_response['entry']):
        if i > 2: break # show just 3 resources per "page"
        resource = entry.get('resource', {})
        print(f'{resource_type}:id', resource.get('id', 'missing'), resource.get('title', '')[:60])
    if page_count > 2: break # pull 3 pages at most to make testing fast
    json_response = get_next_as_raw_json(json_response)
    page_count += 1

GET http://hapi.fhir.org/baseR4/ResearchStudy?_format=json
ResearchStudy:id 2492775 Double blind, placebo-controlled trial of a new class of art
ResearchStudy:id 1164317 Adjuvant Aspirin Treatment in PIK3CA Mutated Colon Cancer Pa
ResearchStudy:id 1164321 A Phase III, Multicenter, Randomized, Open-Label Study Compa
GET http://hapi.fhir.org/baseR4?_getpages=8b604b8c-a8f4-4243-9c39-09175c6cb28d&_getpagesoffset=20&_count=20&_format=json&_pretty=true&_bundletype=searchset
ResearchStudy:id 2111275 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2111271 Safety and Efficacy of the Xanomeline Transdermal Therapeuti
ResearchStudy:id 2085795 Patientenbefragung Augenklinik
GET http://hapi.fhir.org/baseR4?_getpages=8b604b8c-a8f4-4243-9c39-09175c6cb28d&_getpagesoffset=40&_count=20&_format=json&_pretty=true&_bundletype=searchset
ResearchStudy:id 248374 
ResearchStudy:id 247777 
ResearchStudy:id 247565 


In [ ]:
#export
def extract_references_from_resource(resource, field_name):
    "Return a list of references extracted from a single resource and field"
    result = []
    if field_name in resource:
        references = resource[field_name]
        if not isinstance(references, list): references = [references]
        for reference in references:
            _reference = reference.get('reference')
            if _reference is None: continue
            if _reference.startswith('#'): continue
            # TODO: check that we have a relative reference or handle other kinds too
            result.append(_reference)
    return result

TODO: &uarr; we don't need to fetch contained references but we might want to make them bundle entries like the references that we do have to GET - check with Jay if this is already taken care of in FHIR to CDISC

In [ ]:
with open('test/patient_medication_bundle_0c4a1143-8d1c-42ed-b509-eac97d77c9b2.json') as f:
    test_bundle = json.load(f)
test_entry = test_bundle['entry'][3]
test_resource = test_entry['resource'] # resource with medicationReference

In [ ]:
assert (['Medication/bac1387e-3655-4e03-982f-7210faa21ea8'] 
        == extract_references_from_resource(test_resource, 'medicationReference'))

In [ ]:
#export
def extract_references(bundle, field_names):
    "Return a list of relative references e.g. `['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8']`"
    if 'entry' not in bundle: return []
    result = []
    for entry in bundle['entry']:
        resource = entry.get('resource', {})
        for f in field_names:
            result.extend(extract_references_from_resource(resource, f))
    return list(set(result)) # de-duplicate but still return a list

In [ ]:
assert (['Condition/1ddef4ad-fb76-46d6-9f1d-8ed58b173ee8', 'Medication/bac1387e-3655-4e03-982f-7210faa21ea8']
        == sorted(extract_references(test_bundle, ['medicationReference', 'reasonReference'])))

In [ ]:
# show that references are de-duplicated
test_bundle = new_bundle()
test_bundle['entry'].extend([test_entry, test_entry]) # create a bundle with duplicate medication references
assert (['Medication/bac1387e-3655-4e03-982f-7210faa21ea8']
        == extract_references(test_bundle, ['medicationReference', 'reasonReference']))

TODO: We need some config around how many and what type of issues clients can accept 
- some might want to fail if any piece of info cannot be found - i.e. references can't be followed
- some might want to get whatever is available and deal with the inconsistencies

In [ ]:
#export
def get_by_reference(api_base, reference):
    "Return a resource read from a FHIR server by reference, as a list containg a single bundle entry"
    if reference.startswith(api_base):
        reference = reference[len(api_base):].strip('/')
    if reference.startswith('http'):
        print(f'WARNING: Found reference {reference} that does not start with {api_base}')
        return []
    resource_type, id = reference.split('/')
    single_resource = get_as_raw_json(api_base, resource_type, id)
    return [dict(fullUrl = f'{api_base}/{resource_type}/{id}', resource = single_resource)]

If a resource has a reference like
```
'medicationReference': {'reference': 'Medication/bac1387e-3655-4e03-982f-7210faa21ea8'},
```
we can HTTP GET the referenced resource (a `Medication` in this case) with http://hapi.fhir.org/baseR4/Medication/bac1387e-3655-4e03-982f-7210faa21ea8 - then wrap it up as a FHIR `Entry`

No problem if the reference includes the API base. This &darr; works the same as &uarr;
```
'medicationReference': {'reference': 'http://hapi.fhir.org/baseR4/Medication/bac1387e-3655-4e03-982f-7210faa21ea8'},
```

In [ ]:
if api_base == 'http://hapi.fhir.org/baseR4':
    test_id = 'bac1387e-3655-4e03-982f-7210faa21ea8'
    assert get_by_reference(api_base, f'Medication/{test_id}')[0]['resource']['id'] == test_id

GET http://hapi.fhir.org/baseR4/Medication/bac1387e-3655-4e03-982f-7210faa21ea8?_format=json


## Bundle filtering

We need to be able to remove (filter out) entries from the bundle

In [ ]:
#export
def filter_bundle(bundle, filter_fn):
    "Apply a filter function to a bundle in-place"
    bundle['entry'] = [e for e in bundle['entry'] if filter_fn(e)]
    return bundle

In [ ]:
#hide
from nbdev.export import notebook2script
notebook2script()

Converted 00_core.ipynb.
Converted 10_per_patient.ipynb.
Converted 20a_status_filter.ipynb.
Converted 30_cli.ipynb.
Converted 50_web_app.ipynb.
Converted 50a_web_demo.ipynb.
Converted index.ipynb.
